# Uninformed Search

This is the most basic form of seach.It implies trying to reach a goal state with no knowledge of where such goal state is.

Threfore, every possible strategy will just move from one node to another and check whether we reached the goal state or not. It will keep expanding until we reach the goal state.

We have an intrinsic measure of distance (or better, a *cost*) within a graph of nodes and paths, even if the absence of knowledge regarding the gold state: the number of steps associated to a given path. We can also consider adding other measure of distance given for example by the number of kilometers between the start state and our current state.

Given a graph with a cost defined for the paths, we can imagine 3 ways of searching.

- **Breadth-First Search (BFS)**: We start out with the root node and then expand the so-called frotier of examined nodes by considering other nodes that have the same *depth*. Here, the same depth means that the distance between the root node and each of the next-to-consider nodes is the same. In terms of the picture of a tree, this implies considering first expansions of the same depth. When there are ties (i.e. two nodes are one step away from current node) we can just pick one at random or use some rule of thumb.

- **Depth-First Search (DFS)**: TODO:COMPLETE


![BFSvsDFS](./DFSBFS.png)

The implementation of Uninformed Search in pseudocode is a follows:


```function Graph.Search(problem):
  frontier = {[initial]}; explored={}
  loop:
    if frontier is empty: return FAIL
    path = remove.choice(frontier)
    s = path.end; add s to explored
    if s is a goal: return path
    for a in actions:
        add[path + a -> Result(s,a)]
        to frontier
        unless Result(s,a) in frontier or explored
```

So we first have a frontier initialized with with the root node as the only node. Then, we remove a path from the frontier and consider the state `s` that the removed path leads to. Then we add `s` to the explored list. If `s` is the goal state, we return the path. If not, we add to the frontier all the paths that can be constructed starting from `s` and adding each and all of the possible actions `a` (we do this unless `s+a` is in the explored list or is already in the frontier).

The difference between BFS and DFS is simply the path that is removed from the frontier at each iteration, i.e. the particular function or routine that we apply the line `path = remove.choice(frontier)` above. 


## Graph

In [6]:
class GraphNode(object):
    def __init__(self, val):
        self.value = val
        self.children = []
        
    def add_child(self, new_node):
        self.children.append(new_node)
    
    def remove_child(self, del_node):
        if del_node in self.children:
            self.children.remove(del_node)

class Graph(object):
    def __init__(self, node_list):
        self.nodes = node_list
        
    def add_edge(self, node1, node2):
        if(node1 in self.nodes and node2 in self.nodes):
            node1.add_child(node2)
            node2.add_child(node1)
            
    def remove_edge(self, node1, node2):
        if(node1 in self.nodes and node2 in self.nodes):
            node1.remove_child(node2)
            node2.remove_child(node1)

In [7]:
# create a graph
nodeG = GraphNode('G')
nodeR = GraphNode('R')
nodeA = GraphNode('A')
nodeP = GraphNode('P')
nodeH = GraphNode('H')
nodeS = GraphNode('S')

graph1 = Graph([nodeS, nodeH, nodeG, nodeP, nodeR, nodeA]) 
graph1.add_edge(nodeG, nodeR)
graph1.add_edge(nodeA, nodeR)
graph1.add_edge(nodeA, nodeG)
graph1.add_edge(nodeR, nodeP)
graph1.add_edge(nodeH, nodeG)
graph1.add_edge(nodeH, nodeP)
graph1.add_edge(nodeS, nodeR)

## Breadth-First Search

For this algorithm, the frontier is better to be represented with a FIFO (first-in-first-out) queue, which in python can be a usual queue or even a list

In [16]:
from queue import Queue
namequeue = Queue()
# Add elements
namequeue.put("Alice")
namequeue.put("Bob")
namequeue.put("Charlie")

# Remove elements
print(namequeue.get())
print(namequeue.get())
print(namequeue.get())

Alice
Bob
Charlie


In [4]:
namelist = []
namelist.append("Charlie")
namelist.append("Bob")
namelist.append("Alice")

# Remove elements
print(namelist.pop())
print(namelist.pop())
print(namelist.pop())

Alice
Bob
Charlie


Now we moving through the above graph using BFT. Implement the `bfs_search` to return the `GraphNode` with the value `search_value` starting at the `root_node`.

In [18]:
def bfs_search(root_node, search_value):
    bfs_frontier = Queue()
    bfs_frontier.put(root_node)
    bfs_explored = {}

    while True:
        if bfs_frontier.empty():
            return False

        # get (shallowest) node from frontier
        # ensured to be shallowest as frontier is FIFO
        current_node = bfs_frontier.get()
        # note: we don't check gold state here!
        bfs_explored.add(current_node)

        # consider the result of each possible action on 
        # the current node
        for next_node in current_node.children:
            # bfs_frontier is not iterable but bfs_frontier.queue is
            if (
                (next_node not in bfs_explored) and (next_node not in bfs_frontier.queue) 
                ):
                if next_node.value == search_value:
                    # return child node if condition is satisfied
                    # next_node is goal
                    return next_node
                # add child nodes to the frontier
                bfs_frontier.put(next_node)


In [19]:
assert nodeA == bfs_search(nodeS, 'A')
assert nodeS == bfs_search(nodeP, 'S')
assert nodeR == bfs_search(nodeH, 'R')

### Faster solution (Udacity)

In [ ]:
# Solution
def bfs_search(root_node, search_value):
    visited = set()                           # Sets are faster while lookup. Lists are faster to iterate.
    queue = [root_node]
    
    while len(queue) > 0:
        current_node = queue.pop(0)
        visited.add(current_node)

        if current_node.value == search_value:
            return current_node

        for child in current_node.children:
            if child not in visited:          # Lookup
                queue.append(child)
